# Methods for Segmentation in Music Structure Analysis (MSA)
inspired by Foundations of Music Processing by Meinard Müller

The general goal is to divide a given music represantation into temporal segments that correspond to musical parts and to group these segments into musically meaningful categories.

To this end, one needs to convert the waveform into a suitable feature representation that captures musical properties relevant for the structure of interest.

# Problem Definition

Let's define the problem:

Segmentation typically refers to the process of partitioning a given document into multiple segments with the goal of simplifying the representation into something that is more meaningful and easier to analyze than the original document.

For example: 
- in **image processing** the goal is to partition a given image into a set of regions such that each region is similar with respect to some characteristic such as color, intensity, or texture.
- in **music**, the segmentation task is to decompose a given audio stream into acoustically meaningful sections each corresponding to a continuous time interval that is specified by a start and end boundary.

At a *fine level*, the segmentation may aim to find the boundaries between individual notes or to find the beat intervals specified by beat positions. 
At a *coarser level*, the goal may be to detect changes in instrumentation or harmony or to **find the boundaries between verse and chorus sections**.

# Setting up the Environment

### Aspects to Consider

There are different aspects to consider:
1. the complexity of the problem depends on the **kind of music representation** to be analyzed
2. there are various principles including **homogeneity, repetition, and novelty** that a segmentation may be based on
3. one also has to account for **different musical dimensions**, such as melody, harmony, rhythm, or timbre.
4. the segmentation and structure largely depend on the **musical context** and the temporal hierarchy to be considered.

## Libraries needed

Let's start by importing the necessary libraries

In [ ]:
# import libraries

## Importing the Audio File


We will choose as example for this study the *Hungarian Dance No. 5 by Johannes Brahms*, a classic in the music literature and in the field of music structure analysis, in the Orchestral version by Ormandy and the Philadelphia Orchestra.

This piece is part of a set of 21 dance tunes composed by Brahms up to 1869 and based mostly on traditional Hungarian themes.

In [ ]:
# load audio file

## Structure Annotations

The musical structure as indicated in the figure is $A_1A_2B_1B_2CA_3B_3B_4D$, which consists of three repeating A-parts, four repeating B-parts, as well as a C-part and a short closing D-part.

The overall musical structure of this piece can be explained in terms of repeating elements.
A novelty-based segmentation procedure using tempo cues may be used to reveal the substructures of the C-part, whereas a homogeneity-based segmentation procedure using harmonic properties may be suited to distinguish the C-part from the other parts.

In [1]:
# show structure of audio file

## Feature Representations

The first step in automated structure analysis is to transform the given music recording into a suitable feature representation that captures the relevant musical properties. 

There are different feature representations that can be used for segmentation, such as:
- chroma features, related to harmonic and melodic properties
- mel-frequency cepstral coefficients (MFCCs), related to instrumental timbre
- tempogram, related to rhythmic properties


In [2]:
# show comparison of features with better representations

# Segmentation Methods

## Self-Similarity Matrix

To study musical structures and their mutual relations, one general idea is to convert the music signal into a suitable feature sequence and then to compare each element of the feature sequence with all other elements of the sequence, resulting in a self-similarity matrix.

If the feature sequence captures musical properties that stay somewhat constant over the duration of an entire musical part, each of the feature vectors is similar to all other feature vectors within this segment. As a result, an entire block of large values appears in the SSM. In other words, homogeneity properties correspond to block-like structures.

If the feature sequence contains two repeating subsequences (e.g., two segments corresponding to the same melody), the corresponding elements of the two subsequences are similar to each other. As a result, a path (or stripe) of high similarity running parallel to the main diagonal becomes visible in the SSM. In other words, repetitive properties correspond to path-like structures.

In [3]:
# SSM with right cmap

In [ ]:
# SSM comparison in 2 x 2 grid

### SSM based on Chroma Features

Actually, the SSM obtained in this case resembles the idealized SSM to a large extent. The block-like structures corresponding to  
A-part segments indicate that these segments are quite homogeneous with respect to harmony. The same holds for the C-part segment. Furthermore, the small similarity values outside the C-part block (i.e., all cells relating the C-part frames to frames of other segments) show that the C-part segment is harmonically more or less unrelated to all other parts. For the B-part segments, there are path-like structures and no block-like structures. This shows that the B-part segments share the same harmonic progression (i.e., are repetitions with regard to harmony), but are not homogeneous with respect to harmony.

In [4]:
# SSM with Chroma

Most computational approaches to music structure analysis exploit path- and block-like structures of SSMs in one way or another, and the overall algorithmic pipelines typically contain the following general steps:

- The music signal is transformed into a suitable feature sequence.
- A self-similarity matrix is computed from the feature sequence based on a similarity measure.
- Blocks and paths of high overall score are derived from the SSM. Each block or path defines a pair of similar segments.
- Entire groups of mutually similar segments are formed from the pairwise relations by applying a clustering step.
The last step can be considered as forming a kind of transitive closure of the pairwise segment relations induced by block and path structures.

## Path Enhancements

Let's see some strategies to enhance the path structures in the SSM.

### Smoothing the SSM

In [ ]:
# code from feature smoothing, unisci light and strong

### Median Filtering

In [ ]:
# code from feature smoothing

### Diagonal Smoothing

In SSM can appear paths of high similarity that are not perfectly aligned with the main diagonal. This can be due to small timing deviations between the repetitions. To enhance the path structures, one can apply a diagonal smoothing operation to the SSM.

Noise can be reduced simply by using longer analysis windows in the feature computation step and adjusting the feature rate, applying some kind of smoothing filter along the direction of the main diagonal, resulting in an emphasis of diagonal information and a suppression of off-diagonal information.

In [5]:
# code from https://audiolabs-erlangen.de/resources/MIR/FMP/C4/C4S2_SSM-PathEnhancement.html

### Multiple Filtering

To deal with non-diagonal path structures due relative tempo differences, one can apply multiple diagonal smoothing operations with different smoothing strengths and then combine the results.

The main idea of this approach to filter a similarity matrix along various directions that lie in a neighborhood of the direction defined by the main diagonal. Each such direction corresponds to a tempo difference and results in a separate filtered similarity matrix. The final similarity matrix is obtained by taking the cell-wise maximum over all these matrices.

In [ ]:
# code from https://audiolabs-erlangen.de/resources/MIR/FMP/C4/C4S2_SSM-PathEnhancement.html

### Forward-Backward Smoothing

To avoid fading artifacts at the beginning and end of the path structures, one can apply a forward-backward smoothing operation to the SSM, applying the smoothing filter in both directions and taking the cell-wise maximum of the two results.

In [ ]:
# code from https://audiolabs-erlangen.de/resources/MIR/FMP/C4/C4S2_SSM-PathEnhancement.html

### Thresholding

To extract the path structures from the SSM, one can apply a thresholding operation to the SSM, keeping only the cells with values above a certain threshold. This results in a binary matrix that contains only the path structures.

In [6]:
# code from https://audiolabs-erlangen.de/resources/MIR/FMP/C4/C4S2_SSM-Thresholding.html

## Novelty-Based Segmentation

The objective of novelty-based structure analysis is to locate points in time where musical changes occur, thus marking the transition between two subsequent structural parts.

Often a homogeneous segment is followed by another homogeneous segment that stands in contrast to the previous one in terms of instruments or harmony.

Contrasting homogeneous sections are reflected by a local **checkerboard-like block structure** in the SSM.

### Novelty function

The idea of Foote's procedure is to measure local changes by correlating a small checkerboard-like kernel along the main diagonal of an SSM. This results in a novelty function that reveals a peak at time positions where the kernel meets a transition between two contrasting blocks.

In [10]:
# code from https://audiolabs-erlangen.de/resources/MIR/FMP/C4/C4S4_NoveltySegmentation.html

# kernel

# novelty curve

In [11]:
# TODO: check other type of Segmentation types